In [ ]:
import pandas as pd
import re
import os
import logging
from pathlib import Path
import base64
from IPython.display import HTML
from typing import List
from IPython.display import FileLink
import uuid

# --- Setup basic logging ---
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [ ]:
def natural_sort_key(s: str) -> int:
    """A helper function to allow for natural sorting of column names like 'Class 10' after 'Class 2'."""
    match = re.search(r'Class (\d+)', s)
    return int(match.group(1)) if match else -1

def create_download_link(filename: str, text: str = "Download the merged file!") -> HTML:
    """
    Generates a more robust link to download a file from the Jupyter server.
    This version uses a unique ID to prevent conflicts if run multiple times.
    """
    with open(filename, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()

    # Generate a unique ID for the link
    link_id = f"download-link-{uuid.uuid4()}"

    return HTML(f"""
        <a id="{link_id}" download="{filename}" href="data:text/csv;base64,{encoded}">
            {text}
        </a>
        <script>
            // Use a timeout to ensure the link is rendered before clicking
            setTimeout(function() {{
                document.getElementById('{link_id}').click();
            }}, 100);
        </script>
    """)

def cleanup_source_files(files_to_keep: List[str] = None):
    """
    Deletes all .csv files in the current directory except for those
    in the provided keep list. This is a destructive action.
    """
    if files_to_keep is None:
        files_to_keep = []
        
    logging.warning("--- Starting CSV file cleanup ---")
    deleted_count = 0
    for p in Path.cwd().glob('*.csv'):
        if p.is_file() and p.name not in files_to_keep:
            try:
                p.unlink()
                logging.info(f"  - 🗑️ Deleted source file: {p.name}")
                deleted_count += 1
            except Exception as e:
                logging.error(f"Could not delete {p.name}: {e}")
    logging.warning(f"--- Cleanup complete. Deleted {deleted_count} CSV file(s). ---")


def run_final_grade_update():
    """
    Finds and merges all Canvas and iClicker files, preserving official bCourses
    column names and sorting them naturally before calculating the final total.
    """
    # Clean up previous runs
    for p in Path.cwd().glob('updated_*'):
        if p.is_file(): p.unlink()

    # --- 1. File Discovery (Using pathlib) ---
    try:
        current_dir = Path.cwd()
        bc_filenames = sorted([f.name for f in current_dir.glob('*_Grades-ENGIN_7.csv')], reverse=True)
        ic_filenames = sorted([f.name for f in current_dir.glob('iClicker_GradesExport_Canvas_*.csv')])

        if not bc_filenames:
            raise StopIteration("No Canvas gradebook files found matching the pattern.")

        logging.info(f"✅ Found {len(bc_filenames)} Canvas gradebooks and {len(ic_filenames)} iClicker files.")
        
    except StopIteration as e:
        logging.error(f"❌ Critical: Could not find required CSV files. {e}")
        return

    # --- 2. Build a Master Map of Official Column Names ---
    poll_column_map = {}
    for filename in bc_filenames:
        try:
            df = pd.read_csv(filename, dtype=str, nrows=0) # Read only headers for efficiency
            for col in df.columns:
                match = re.match(r'(Class \d+ - Poll)( \(\d+\))', col)
                if match:
                    simple_name = match.group(1)
                    poll_column_map[simple_name] = col # Map 'Class 1 - Poll' -> 'Class 1 - Poll (12345)'
        except Exception as e:
            logging.warning(f"Could not read headers from {filename}: {e}")
    logging.info(f"Built a map of {len(poll_column_map)} official poll column names.")
    
    # --- 3. Load and Prepare a Master Gradebook ---
    try:
        master_bc_filename = bc_filenames[0]
        master_df = pd.read_csv(master_bc_filename, dtype=str)

        master_total_col = next((col for col in master_df.columns if re.match(r'iClicker \(Total\) \(\d+\)', col)), None)
        if not master_total_col:
            logging.error("❌ Critical: Could not find the 'iClicker (Total) (number)' column.")
            return

        is_student = pd.to_numeric(master_df['SIS User ID'], errors='coerce').notna()
        student_data = master_df[is_student].copy()
        special_rows = master_df[~is_student].copy()
        
        is_test_student = student_data['Student'] == 'Student, Test'
        test_student_row = student_data[is_test_student].copy()
        student_data = student_data[~is_test_student].copy()

    except Exception as e:
        logging.error(f"An error occurred loading the main gradebook: {e}")
        return

    # --- 4. Consolidate ALL Source Files into the Master Record ---
    all_source_files = bc_filenames + ic_filenames
    processed_poll_columns = set()
    iclicker_poll_columns = set() 
    for filename in all_source_files:
        try:
            source_df = pd.read_csv(filename, dtype=str)
            source_students = source_df[pd.to_numeric(source_df['SIS User ID'], errors='coerce').notna()].copy()
            
            source_poll_col_name = next((col for col in source_students.columns if 'Class' in col and 'Poll' in col), None)
            if not source_poll_col_name: continue

            simple_poll_name = re.match(r'Class \d+ - Poll', source_poll_col_name).group(0)
            master_poll_col_name = poll_column_map.get(simple_poll_name, simple_poll_name)
            processed_poll_columns.add(master_poll_col_name)

            if filename in ic_filenames:
                iclicker_poll_columns.add(master_poll_col_name)

            if master_poll_col_name not in student_data.columns:
                student_data[master_poll_col_name] = 0.0

            source_subset = source_students[['SIS User ID', source_poll_col_name]].rename(columns={source_poll_col_name: 'New_Score'})
            student_data = pd.merge(student_data, source_subset, on='SIS User ID', how='left')
            
            # Combine scores, taking the max between existing and new
            student_data[master_poll_col_name] = pd.to_numeric(student_data[master_poll_col_name], errors='coerce').fillna(0)
            student_data['New_Score'] = pd.to_numeric(student_data['New_Score'], errors='coerce').fillna(0)
            student_data[master_poll_col_name] = student_data[[master_poll_col_name, 'New_Score']].max(axis=1)
            
            student_data.drop(columns=['New_Score'], inplace=True)

        except Exception as e:
            logging.error(f"  - ❌ An error occurred while processing {filename}: {e}")
            continue

    # --- 5. Final Calculation and Reconstruction ---
    sorted_poll_columns = sorted(list(processed_poll_columns), key=natural_sort_key)
    logging.info(f"Final poll columns in order: {sorted_poll_columns}")

    logging.info(f"🔄 Adding scores from iClicker files to '{master_total_col}'...")
    
    # Ensure the existing total and new poll columns are numeric before calculation
    student_data[master_total_col] = pd.to_numeric(student_data[master_total_col], errors='coerce').fillna(0)
    for col in iclicker_poll_columns:
        if col in student_data.columns:
            student_data[col] = pd.to_numeric(student_data[col], errors='coerce').fillna(0)
    
    sorted_iclicker_cols = sorted(list(iclicker_poll_columns), key=natural_sort_key)
    
    if sorted_iclicker_cols:
        logging.info(f"Columns from iClicker files to be added: {sorted_iclicker_cols}")
        new_iclicker_sum = student_data[sorted_iclicker_cols].sum(axis=1)
        # Add the sum of the new scores to the existing total
        student_data[master_total_col] += new_iclicker_sum
    else:
        logging.info("No new iClicker columns found to add. Existing total preserved.")

    # Round the final calculated total
    student_data[master_total_col] = student_data[master_total_col].round(2)
    
    # Recombine the DataFrame parts
    final_df = pd.concat([special_rows, student_data, test_student_row], ignore_index=True)

    # --- 6. Save Final Truncated File ---
    cols_to_keep = list(master_df.columns[:5]) + sorted_poll_columns + [master_total_col]
    cols_to_keep = [col for col in cols_to_keep if col in final_df.columns]
    
    final_df_truncated = final_df[cols_to_keep].copy()

    # Move test student to the end dynamically
    is_test = final_df_truncated['Student'] == 'Student, Test'
    if is_test.any():
        test_row_data = final_df_truncated[is_test]
        final_df_truncated = final_df_truncated[~is_test]
        final_df_truncated = pd.concat([final_df_truncated, test_row_data], ignore_index=True)

    # Set first row to be empty
    if not final_df_truncated.empty:
        final_df_truncated.iloc[0] = ""
    
    output_filename = f'updated_{master_bc_filename}'
    final_df_truncated.to_csv(output_filename, encoding='utf-8-sig', index=False)
    
    logging.info(f"\n✨ Done! All files consolidated. Final gradebook saved to '{output_filename}'.")

    # --- 7. Force download of the output file ---
    display(create_download_link(output_filename))
    
    # --- 8. Clean up source files (optional) ---
    cleanup_source_files(files_to_keep=[])

# --- Main execution block ---
# To run the script, call this function in a new cell:
run_final_grade_update()